### **Lesson 2 – Unsupervised Learning (Clustering cơ bản)**

**🔑 Prerequisite**

* Biết array/vector.
* Biết khoảng cách Euclidean (distance).
* Đã quen với loop và numpy.

---

#### ❓ Unsupervised Learning là gì?

* Khác với SL: **không có nhãn (label)**.
* Mục tiêu: tìm **cấu trúc ẩn** trong dữ liệu.
* Ví dụ:

  * Gom nhóm khách hàng có hành vi giống nhau.
  * Gom news articles theo chủ đề mà không cần biết nhãn trước.

---

#### ❓ Clustering là gì?

* **Clustering** = gom dữ liệu thành các nhóm (clusters) sao cho:

  * Các điểm trong cùng 1 cluster thì **gần nhau**.
  * Các điểm khác cluster thì **xa nhau**.

---

#### ❓ K-means hoạt động như thế nào?

Ý tưởng:

1. Chọn **k** (số cluster).
2. Khởi tạo k centroid ngẫu nhiên.
3. Lặp lại:

   * Assign mỗi điểm vào cluster có centroid gần nhất.
   * Update centroid = trung bình của cluster.
4. Kết thúc khi centroid không đổi nhiều nữa.

---

#### ❓ Ưu – Nhược điểm của K-means?

**Ưu**:

* Nhanh, dễ code.
* Hay dùng cho dữ liệu dạng số.

**Nhược**:

* Phải chọn trước k.
* Nhạy cảm với outliers.
* Nếu dữ liệu có hình dạng lạ (không phải cụm tròn), thì k-means không tốt.

---

#### 🚀 Practice Exercise

Implement K-means mini

Dataset toy:

```
points = np.array([
    [1, 2], [1, 4], [1, 0],
    [10, 2], [10, 4], [10, 0]
])
```

👉 Rõ ràng ở đây có 2 nhóm: gần (1,*) và gần (10,*).

Nhiệm vụ:

1. Code k-means (k=2):

   * Khởi tạo centroid ngẫu nhiên từ dataset.
   * Assign points vào cluster gần nhất (dùng Euclidean distance).
   * Update centroid.
   * Chạy vài vòng (10 epochs).
2. In ra **final centroids** + xem points được chia nhóm thế nào.

---

#### 🧩 Solution

In [1]:
# Imports
import numpy as np

In [2]:
# Dataset
points = np.array([[1, 2], [1, 4], [1, 10], [10, 2], [10, 4], [10, 0]], float)
num_points = points.shape[0]

In [3]:
# Parameters
k, epochs = 2, 10
pending: list[int]
centroids: list[list[tuple, float]]
division: list[int] = [0] * num_points
best: tuple[list[tuple], list[int]]

In [4]:
# Functions
def init_centroids():
    global centroids, pending
    pending = list(range(num_points))
    centroids = []
    for i in np.random.choice(num_points, k, replace=False):
        pending.remove(i)
        centroids.append([points[i].copy(), 1])


def dist(a: np.ndarray, b: np.ndarray):
    return np.sqrt(((a - b) ** 2).sum())


def update_centroid(cen: list[int], pt: np.ndarray):
    cen[1] += 1
    cen[0] += (pt - cen[0]) / cen[1]


def loss():
    return sum(
        dist(points[i], points[j])
        for i in range(num_points)
        for j in range(num_points)
        if division[i] == division[j]
    )

In [5]:
# Training
def train_step():
    init_centroids()
    for i in np.random.permutation(pending):
        pt = points[i]
        t = 0
        for j in range(1, k):
            if dist(pt, centroids[j][0]) < dist(pt, centroids[t][0]):
                t = j
        division[i] = t
        update_centroid(centroids[t], pt)


min_loss = 1 << 30
for _ in range(epochs):
    train_step()
    if (l := loss()) < min_loss:
        min_loss = l
        best = ([c for c, _ in centroids], division.copy())

In [6]:
print(f"Final loss: {min_loss}")

print(f"Final centroids: {', '.join(map(str, best[0]))}")
print("Mapping (point) -> (centroid_idx):")
for i in range(num_points):
    print(f"{str(points[i])} -> {best[1][i]}")

Final loss: 48.0
Final centroids: [10.  2.], [1.         5.33333333]
Mapping (point) -> (centroid_idx):
[1. 2.] -> 1
[1. 4.] -> 1
[ 1. 10.] -> 1
[10.  2.] -> 0
[10.  4.] -> 0
[10.  0.] -> 0
